#### 1. Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:  
• Наименование вакансии,  
• Предлагаемую зарплату,   
• Ссылку на саму вакансию  

In [1]:
import pandas as pd
import requests
import json
import re
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm

In [2]:
def get_vacancy_salary(vacancy_link):
    with requests.session() as s:
        r = s.get(vacancy_link)
    html = r.text
    parsed_html = bs(html, 'html.parser')
    meta = parsed_html.find_all('meta', property="og:description")
    text = str(meta[0])
    parsed_text = re.findall(r'зарплата.+\.', text.replace(' ', ''))[0]
    salary = re.findall(r'\d+', parsed_text)
    if len(salary) == 0:
        min_salary, max_salary = 0, 0
    elif len(salary) == 1:
        min_salary = max_salary = salary[0]
    elif len(salary) == 2:
        min_salary, max_salary = salary
    else:
        min_salary, max_salary = 0, 0
    return min_salary, max_salary


def get_vacancy_info(keywords=[], page_count=1):
    if len(keywords) == 0 or page_count < 0:
        return
    else:
        df_list = []
        for keyword in tqdm(keywords):
            print(f'Keyword: {keyword}')
            key = '%20'.join(keyword.split(' ')).capitalize()
            for i in tqdm(range(1, page_count+1)):
                print(f'Page: {i} of {page_count}')
                link = f'https://www.superjob.ru/vacancy/search/?keywords={key}&page={i}'
                with requests.session() as s:
                    r = s.get(link)
                html = r.text
                parsed_html = bs(html, 'html.parser')
                tags = parsed_html.find_all('a', href=True)
                vacancies = {
                    'id': [],
                    'name': [],
                    'min_salary': [],
                    'max_salary': [],
                    'link': []
                }
                for tag in tags:
                    text = re.findall(r'<a class="icMQ_ _1QIBo f-test-link-.+', str(tag))
                    if len(text) > 0:
                        vacancy_id = re.findall(r'(\d+)\.', tag['href'])[0]
                        vacancy_name = bs(text[0]).get_text()
                        vacancy_link = 'https://www.superjob.ru' + tag['href']
                        vacancies['id'].append(vacancy_id)
                        vacancies['name'].append(vacancy_name)
                        vacancy_min_salary, vacancy_max_salary = get_vacancy_salary(vacancy_link)
                        vacancies['min_salary'].append(vacancy_min_salary)
                        vacancies['max_salary'].append(vacancy_max_salary)
                        vacancies['link'].append(vacancy_link)
                    df_temp = pd.DataFrame(vacancies)
                df_list.append(df_temp)
        return pd.concat(df_list).reset_index(drop=True)
    

In [3]:
df = get_vacancy_info(keywords=['программист', 'разработчик'], page_count=3)

Keyword: программист


Page: 1 of 3
Page: 2 of 3
Page: 3 of 3
Keyword: разработчик


Page: 1 of 3
Page: 2 of 3
Page: 3 of 3



In [4]:
df.head(10)

,id,name,min_salary,max_salary,link
0,32363006,Программист 1С,100000,100000,https://www.superjob.ru/vakansii/programmist-1...
1,32253050,Инженер-программист,0,0,https://www.superjob.ru/vakansii/inzhener-prog...
2,32066066,Программист-разработчик 1С,120000,120000,https://www.superjob.ru/vakansii/programmist-r...
3,32171279,Инженер-программист C#,80000,120000,https://www.superjob.ru/vakansii/inzhener-prog...
4,31987085,Программист,0,0,https://www.superjob.ru/vakansii/programmist-3...
5,31614437,Web-программист,70000,100000,https://www.superjob.ru/vakansii/web-programmi...
6,32327168,"Программист Android 4.2 (технологии DI, SQLite)",0,0,https://www.superjob.ru/vakansii/programmist-a...
7,30873935,Программист SQL,90000,90000,https://www.superjob.ru/vakansii/programmist-s...
8,31536337,Программист-стажер,50000,70000,https://www.superjob.ru/vakansii/programmist-s...
9,32407139,Инженер-программист,80000,80000,https://www.superjob.ru/vakansii/inzhener-prog...


#### 2. Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

In [5]:
df = get_vacancy_info(keywords=['Разработчик Python', 'Разработчик Java', 'Разработчик C++'], page_count=3)

Keyword: Разработчик Python


Page: 1 of 3
Page: 2 of 3
Page: 3 of 3
Keyword: Разработчик Java


Page: 1 of 3
Page: 2 of 3
Page: 3 of 3
Keyword: Разработчик C++


Page: 1 of 3
Page: 2 of 3
Page: 3 of 3



In [6]:
df.head(10)

,id,name,min_salary,max_salary,link
0,32360537,Аналитик-разработчик Python,0,0,https://www.superjob.ru/vakansii/analitik-razr...
1,32214239,Разработчик аналитической отчетности,0,0,https://www.superjob.ru/vakansii/razrabotchik-...
2,32357029,Python developer,0,0,https://www.superjob.ru/vakansii/python-develo...
3,29723693,Web-программист / Web-разработчик,170000,170000,https://www.superjob.ru/vakansii/web-programmi...
4,32355383,Web-разработчик,70000,70000,https://www.superjob.ru/vakansii/web-razrabotc...
5,32266222,"Web-программист, Web-разработчик",0,0,https://www.superjob.ru/vakansii/web-programmi...
6,32392624,Web-разработчик,70000,120000,https://www.superjob.ru/vakansii/web-razrabotc...
7,32203665,"Web-разработчик (PHP, YII2)",126500,126500,https://www.superjob.ru/vakansii/web-razrabotc...
8,31478639,Web-разработчик,80000,90000,https://www.superjob.ru/vakansii/web-razrabotc...
9,31035766,Программист на python,0,0,https://www.superjob.ru/vakansii/programmist-n...
